In [1]:
import pyaudio
import struct
import numpy as np
import matplotlib.pyplot as plt
from scipy.fftpack import fft
from scipy import signal

%matplotlib qt



In [2]:
CHUNK = 2400
#CHUNK = 800
FORMAT = pyaudio.paInt16
CHANNELS = 2 # tine minte asta
RATE = 48000

In [3]:
p = pyaudio.PyAudio()

stream = p.open(
    format=FORMAT,
    channels=CHANNELS,
    rate=RATE,
    input=True,
    output=False,
    frames_per_buffer=CHUNK,
    input_device_index=12
)

In [ ]:
fig, (ax, ax2) = plt.subplots(2)
x = np.arange(0, 2 * CHUNK * CHANNELS, 2)
x_fft = np.linspace(0, RATE, CHUNK * 2)

line, = ax.plot(x, np.random.rand(CHUNK * 2), '-', lw=2)
line_fft, = ax2.semilogx(x_fft, np.random.rand(CHUNK * 2), '-', lw=2)
ax.set_ylim(0, 8000)
ax.set_xlim(0, CHUNK * 2)

ax2.set_xlim(20, RATE/4)

while True:
    data = stream.read(CHUNK)
    #data_int = np.array(struct.unpack(str(CHUNK * 2 * CHANNELS) + 'B', data), dtype='b')[::2] + 255
    data_int = np.array(struct.unpack(str(CHUNK * CHANNELS) + 'h', data))
    line.set_ydata(data_int)
    y_fft = fft(data_int)
    line_fft.set_ydata(np.abs(y_fft[0:CHUNK * 2]) * 2 / (1000 * CHUNK * 2))
    fig.canvas.draw()
    fig.canvas.flush_events()
    
#len(data)

In [6]:
host_info = p.get_host_api_info_by_index(0)    
device_count = host_info.get('deviceCount')
devices = []

# iterate between devices:
for i in range(0, device_count):
    device = p.get_device_info_by_host_api_device_index(0, i)
    devices.append(device['name'])

print(devices)

['Microsoft Sound Mapper - Input', 'Microphone Array (Realtek High ', 'Stereo Mix (Realtek High Defini', 'Microsoft Sound Mapper - Output', 'Speakers (Realtek High Definiti']


In [7]:
cock = [p.get_device_info_by_index(i) for i in range(p.get_device_count())]
#print(cock)

for i in range(len(cock)):
    print("{}: {}, input: {}, output: {}, sample: {}".format(cock[i]['index'], cock[i]['name'], cock[i]['maxInputChannels'], cock[i]['maxOutputChannels'], cock[i]['defaultSampleRate']))

0: Microsoft Sound Mapper - Input, input: 2, output: 0, sample: 44100.0
1: Microphone Array (Realtek High , input: 2, output: 0, sample: 44100.0
2: Stereo Mix (Realtek High Defini, input: 2, output: 0, sample: 44100.0
3: Microsoft Sound Mapper - Output, input: 0, output: 2, sample: 44100.0
4: Speakers (Realtek High Definiti, input: 0, output: 2, sample: 44100.0
5: Primary Sound Capture Driver, input: 2, output: 0, sample: 44100.0
6: Microphone Array (Realtek High Definition Audio), input: 2, output: 0, sample: 44100.0
7: Stereo Mix (Realtek High Definition Audio), input: 2, output: 0, sample: 44100.0
8: Primary Sound Driver, input: 0, output: 2, sample: 44100.0
9: Speakers (Realtek High Definition Audio), input: 0, output: 2, sample: 44100.0
10: Speakers (Realtek High Definition Audio), input: 0, output: 2, sample: 48000.0
11: Microphone Array (Realtek High Definition Audio), input: 2, output: 0, sample: 48000.0
12: Stereo Mix (Realtek High Definition Audio), input: 2, output: 0, sampl

In [ ]:
fig, (ax, ax2) = plt.subplots(2)
x = np.arange(0, 2 * CHUNK * CHANNELS, 2)
xf = np.linspace(0, RATE, CHUNK * CHANNELS)
line, = ax.plot(x, np.random.rand(CHUNK * CHANNELS), '-', lw=2)
line_fft, = ax2.plot(xf, np.random.rand(CHUNK * CHANNELS), '-', lw=2)
ax.set_ylim(-1, 1)
ax.set_xlim(0, CHUNK * 2 * CHANNELS)

ax2.set_xlim(0, RATE/4)


while True:
    data = stream.read(CHUNK)
    data_int = (np.array(struct.unpack(str(CHUNK * CHANNELS) + 'h', data))) + 1 
    data_int = data_int / max(abs(data_int))
    line.set_ydata(data_int)

    yf = fft(data_int)
    yf = np.abs(yf / max(yf))
    #line_fft.set_ydata(np.abs(yf[0:CHUNK*CHANNELS]) / (128 * CHUNK * CHANNELS))
    line_fft.set_ydata(yf)
    #del yf
    fig.canvas.draw()
    #plt.pause(0.005)
    fig.canvas.flush_events()
    

In [5]:
fig, ax = plt.subplots(1)
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')
plt.show()
while True:
    data = stream.read(CHUNK)
    data_int = (np.array(struct.unpack(str(CHUNK * CHANNELS) + 'h', data))) + 1
    f, t, Sxx = signal.spectrogram(data_int)
    plt.pcolormesh(t, f, Sxx, shading='gouraud')

    

KeyboardInterrupt: 

In [ ]:
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
import math


In [ ]:
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(
    IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))

currentVolumeDb = volume.GetMasterVolumeLevel()
print(currentVolumeDb)